### PanStarrs

In [ ]:
from astropy.table import Table

In [ ]:
import matplotlib.pyplot as plt
import arya

In [ ]:
from astropy import units as u

In [ ]:
import numpy as np

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../../imaging")

from phot_utils import xmatch


In [ ]:
panstarrs = Table.read("../../survey_data/yasone2_panstarrs.fits")

In [ ]:
filename_g = "../yasone2/img_g_01/nobkg-cat.fits"
filename_r = "../yasone2/img_r_01/nobkg-cat.fits"
filename_i = "../yasone2/img_i_01/nobkg-cat.fits"


In [ ]:
cat_g = Table.read(filename_g)
cat_r = Table.read(filename_r)
cat_i = Table.read(filename_i)

In [ ]:
ra0 = np.median(cat_r["ra"])
dec0 = np.median(cat_r["dec"])

In [ ]:
plt.figure()
plt.scatter(cat_r["ra"], cat_r["dec"], s=1, color="r")
plt.scatter(cat_g["ra"], cat_g["dec"], s=0.8, color="g")
plt.scatter(cat_i["ra"], cat_i["dec"], s=0.6, color="y")

plt.scatter(panstarrs["ra"], panstarrs["dec"], s=0.2)
plt.xlim(ra0 - 0.1, ra0+0.1)
plt.ylim(dec0 - 0.1, dec0 + 0.1)

In [ ]:
cat_g["ra"]

In [ ]:
panstarrs["dec"].data

In [ ]:
xmatch_idx, xmatch_dist, xmatch_filt_panstarrs = xmatch(cat_r["ra"], cat_r["dec"], panstarrs["ra"].reshape(-1), panstarrs["dec"].reshape(-1), 3*u.arcsec)

In [ ]:
xmatch_g_idx, xmatch_g_dist, xmatch_g_filt = xmatch(cat_r["ra"], cat_r["dec"], cat_g["ra"], cat_g["dec"], 3*u.arcsec)
xmatch_i_idx, xmatch_i_dist, xmatch_i_filt = xmatch(cat_r["ra"], cat_r["dec"], cat_i["ra"], cat_i["dec"], 3*u.arcsec)

In [ ]:
xmatch_filt = xmatch_filt_panstarrs & xmatch_g_filt & xmatch_i_filt

In [ ]:
panstarrs_xmatch = panstarrs[xmatch_idx]
osiris_x_panstarrs = cat_r[xmatch_filt]

In [ ]:
cat = cat_r.copy()
cat["mag_g"] = cat_g["mag_2"][xmatch_g_idx]
cat["mag_i"] = cat_i["mag_2"][xmatch_i_idx]
cat["mag_r"] = cat["mag_2"]

for filt in ["g", "r", "i"]:
    cat[f"mag_{filt}_panstarrs"] = panstarrs_xmatch[f"{filt}MeanPSFMag"].reshape(-1)
    cat[f"mag_{filt}_err_panstarrs"] = panstarrs_xmatch[f"{filt}MeanPSFMagErr"].reshape(-1)

    cat[f"mag_{filt}_panstarrs"][xmatch_filt_panstarrs] = np.nan
    cat[f"mag_{filt}_err_panstarrs"][xmatch_filt_panstarrs] = np.nan
        

In [ ]:
plt.figure()
plt.scatter(panstarrs_xmatch["ra"], panstarrs_xmatch["dec"])
plt.scatter(cat["ra"][xmatch_filt]/u.degree, cat["dec"][xmatch_filt]/u.degree, s=1)
# plt.xlim(-0.1, 0.1)
# plt.ylim(-0.1, 0.1)

In [ ]:

fig, axs = plt.subplots(2, 3, sharex=True, sharey="row", figsize=(7, 4))

for i in range(3):
    plt.sca(axs[0][i])

    filt = ["g", "r", "i"][i]
    mag0 = cat[f"mag_{filt}"]
    mag_ps = cat[f"mag_{filt}_panstarrs"]
    mag_ps_err = cat[f"mag_{filt}_err_panstarrs"]
    plt.scatter(mag0, mag_ps, s=2)
    plt.xlim(26, 15)
    plt.ylim(26, 15)
    plt.plot([26, 10], [26, 10], color="black")
    if i == 0:
        plt.ylabel(f"{filt} (PS)")
    
    plt.sca(axs[1][i])
    
    yerr = np.maximum(mag_ps_err, 0)
    plt.errorbar(mag0, mag_ps-mag0, yerr=yerr, fmt=".")
    
    plt.ylim(3, -3)
    plt.axhline(0, color="black")
    
    plt.xlabel(f"{filt} (osiris)")

plt.tight_layout()

# Catalogue creation